# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name(s): Emmanuel Macario
###### Python version: 3.6.0

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the seven functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [42]:
# Import useful libraries
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [43]:
# Filename constants used for easy file access
CSV1 = 'breast-cancer-dos.csv'
CSV2 = 'car-dos.csv'
CSV3 = 'hypothyroid-dos.csv'
CSV4 = 'mushroom-dos.csv'


def preprocess(filename):
    """
    Opens a data file in csv, and transforms it into
    a usable format.
    """
    
    # Read csv data into a dataframe. Assign
    # an integer value to each attribute in
    # the data.
    df = pd.read_csv(filename, header=None)
    
    
    
    # Partition the data into instances and
    # instance class labels.
    #instance_list = df.iloc[:,:-1]
    #class_list = df.iloc[:,-1]
    #data_set = instance_list, class_list
    
    return df


# Test preprocess function
df = preprocess(CSV2)
df.head(5)


,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [44]:
# This function should build a supervised NB model
def train_supervised(df):
    """
    Builds a supervised Naive Bays model,
    given a preprocessed set of training
    data, by calculating counts from the
    training data.
    
    Inputs: a training set of data, consisting
    of a list of instances, and a list of class 
    labels for those instances.
    
    Outputs: 2-tuple containing a class frequency 
    dictionary and also the supervised model.
    """
    
    # Dictionary contains the frequencies
    # of each class in the training set
    class_freqs = defaultdict(int) 
    
    # The supervised Naive Bays model
    supervised_model = {}
    
    # For every instance in the training
    # data, update the model
    for instance in df.values:
        
        # Get the instance's class label
        class_label = instance[-1]
        
        
        # Update the class frequency dictionary
        class_freqs[class_label] += 1
        
        
        # If the class label is not already in
        # the model, create a nested set of
        # dictionaries for the class.
        if class_label not in supervised_model:
            
            # New dictionary for every new class
            supervised_model[class_label] = {}
            
            # For each attribute, initialise
            # a new default dictionary.
            for attr in df.columns[:-1]:
                supervised_model[class_label][attr] = defaultdict(int)
            
            
        # For every attribute in the instance,
        # get its corresponding value and update
        # the model.
        for attr in df.columns[:-1]:
            attr_value = instance[attr]
            supervised_model[class_label][attr][attr_value] += 1
    
    
    return class_freqs, supervised_model



# Test function
class_freqs, supervised_model = train_supervised(df)


supervised_model

{'acc': {0: defaultdict(int,
              {'high': 108, 'low': 89, 'med': 115, 'vhigh': 72}),
  1: defaultdict(int, {'high': 105, 'low': 92, 'med': 115, 'vhigh': 72}),
  2: defaultdict(int, {'2': 81, '3': 99, '4': 102, '5more': 102}),
  3: defaultdict(int, {'4': 198, 'more': 186}),
  4: defaultdict(int, {'big': 144, 'med': 135, 'small': 105}),
  5: defaultdict(int, {'high': 204, 'med': 180})},
 'good': {0: defaultdict(int, {'low': 46, 'med': 23}),
  1: defaultdict(int, {'low': 46, 'med': 23}),
  2: defaultdict(int, {'2': 15, '3': 18, '4': 18, '5more': 18}),
  3: defaultdict(int, {'4': 36, 'more': 33}),
  4: defaultdict(int, {'big': 24, 'med': 24, 'small': 21}),
  5: defaultdict(int, {'high': 30, 'med': 39})},
 'unacc': {0: defaultdict(int,
              {'high': 324, 'low': 258, 'med': 268, 'vhigh': 360}),
  1: defaultdict(int, {'high': 314, 'low': 268, 'med': 268, 'vhigh': 360}),
  2: defaultdict(int, {'2': 326, '3': 300, '4': 292, '5more': 292}),
  3: defaultdict(int, {'2': 576, '4'

In [45]:

def predict_supervised(df, class_freqs, supervised_model):
    """
    Predicts the class labels for a set of
    instances, based on a supervised Naive Bayes
    model.
    """
    
    # A list of predictions for each instance
    prediction_list = []
    
    # Calculate the total number of instances
    total_instances = sum(class_freqs.values())
    
    # For each instance, predict the class
    # based on our model.
    for instance in df.values:
        
        max_prob = 0
        predicted_class = ""
        
        # Test each class label to see what the predicted class
        for class_label in class_freqs:
            
            # The prior probability of a class
            prob = class_freqs[class_label]/total_instances
            
            
            # Multiply the prior probability of the predicted class
            # by the posterior probability of the instance attribute
            # value, given the class.
            for attr in supervised_model[class_label]:
                prob *= (supervised_model[class_label][attr][instance[attr]]/class_freqs[class_label])
                
            
            # If the probability is the highest seen
            # thus far, set the predicted class to the
            # class label.
            if prob > max_prob:
                max_prob = prob
                predicted_class = class_label
            
        
        # Add the predicted class to our list
        prediction_list.append(predicted_class)
        
    return prediction_list


# Test the function
prediction_list = predict_supervised(df, class_freqs, supervised_model)

In [46]:

def evaluate_supervised(prediction_list, class_list):
    """
    Evaluates a set of predictions, in a supervised
    context. Uses accuracy as the primary method of
    evaluation.
    """
    
    # Validation checking
    assert(len(prediction_list) == len(class_list))
    
    # Calculate and return the accuracy of the model
    return (prediction_list == class_list).value_counts().loc[True]/len(prediction_list)


# Test the function
class_list = df.iloc[:,-1]
accuracy = evaluate_supervised(prediction_list, class_list)
print(accuracy)

0.8738425925925926


In [47]:

def train_unsupervised(df):
    """
    Builds an unsupervised Naive Bayes model,
    frm a given set of unlabelled training data.
    """
    
    # Get a list of the unique class labels
    class_labels = df.iloc[:,-1].unique()
    
    # The list containing random class
    # distributions for each instance.
    random_class_distributions = []
    
    
    # For each training instance, create
    # a non-uniform class distribution for
    # that instance.
    for _ in df.values:
        
        random_class_distributions.append(
            np.random.dirichlet(np.ones(len(class_labels)),size=1)[0])
        
    
    # Make a new dataframe consisting of random class distributions
    # for each instance.
    new_df = pd.DataFrame(random_class_distributions, 
                          columns=class_labels)   
        
    
    # Fuck the class labels mate
    df = df.iloc[:,:-1]
    
    
    # Dictionary contains the (random) frequencies
    # of each class in the training set
    class_freqs = defaultdict(int) 
    
    
    # Count the frequencies
    for class_label in class_labels:
        class_freqs[class_label] = new_df[class_label].sum()
    
    
    # Initialise the unsupervised Naive Bays model, which
    # is just a dictionary of dictionaries of dictionaries
    unsupervised_model = {c:{i:defaultdict(float) for i in range(df.shape[1])} for c in class_labels}
    
    
    # Concatenate the dataframes
    df2 = pd.concat([df, new_df], axis=1)
    
    
    # For every instance in the training
    # data, update the unsupervised model
    for i in range(len(df)):
        
        for attr in df.columns:
            
            for class_label in class_labels:
                
                unsupervised_model[class_label][attr][df2[attr][i]] += df2[class_label][i]
                
                
    return class_freqs, unsupervised_model, df2



# Test the function
cf, um, df = train_unsupervised(df)
cf

defaultdict(int,
            {'acc': 427.3342786348936,
             'good': 435.1462251193486,
             'unacc': 425.45979754482875,
             'vgood': 440.0596987009288})

In [48]:
sum(cf.values())

1727.9999999999998

In [49]:
um

{'acc': {0: defaultdict(float,
              {'high': 111.27096510674892,
               'low': 100.22243357453418,
               'med': 108.0397881323836,
               'vhigh': 107.80109182122655}),
  1: defaultdict(float,
              {'high': 106.49507117230705,
               'low': 112.19563259968781,
               'med': 104.23510349062283,
               'vhigh': 104.40847137227563}),
  2: defaultdict(float,
              {'2': 104.19083659788046,
               '3': 108.17990425127823,
               '4': 105.44920523679052,
               '5more': 109.51433254894404}),
  3: defaultdict(float,
              {'2': 146.94564627393842,
               '4': 142.76098235223327,
               'more': 137.62765000872147}),
  4: defaultdict(float,
              {'big': 143.21818998270382,
               'med': 142.87558501370114,
               'small': 141.2405036384883}),
  5: defaultdict(float,
              {'high': 143.79370975527334,
               'low': 144.26594543988165,

In [53]:
cf

defaultdict(int,
            {'acc': 427.3342786348936,
             'good': 435.1462251193486,
             'unacc': 425.45979754482875,
             'vgood': 440.0596987009288})

In [55]:
df.head(10)

,0,1,2,3,4,5,unacc,acc,vgood,good
0,vhigh,vhigh,2,2,small,low,0.013408,0.607227,0.064997,0.314367
1,vhigh,vhigh,2,2,small,med,0.038113,0.127509,0.678461,0.155918
2,vhigh,vhigh,2,2,small,high,0.097038,0.479029,0.082677,0.341256
3,vhigh,vhigh,2,2,med,low,0.266023,0.093445,0.015448,0.625083
4,vhigh,vhigh,2,2,med,med,0.058349,0.344024,0.088702,0.508926
5,vhigh,vhigh,2,2,med,high,0.586590,0.083878,0.047782,0.281750
6,vhigh,vhigh,2,2,big,low,0.072204,0.108967,0.624145,0.194684
7,vhigh,vhigh,2,2,big,med,0.046892,0.219687,0.135917,0.597504
8,vhigh,vhigh,2,2,big,high,0.477617,0.006381,0.049854,0.466148
9,vhigh,vhigh,2,4,small,low,0.005919,0.122032,0.335841,0.536208


In [56]:
um

{'acc': {0: defaultdict(float,
              {'high': 111.27096510674892,
               'low': 100.22243357453418,
               'med': 108.0397881323836,
               'vhigh': 107.80109182122655}),
  1: defaultdict(float,
              {'high': 106.49507117230705,
               'low': 112.19563259968781,
               'med': 104.23510349062283,
               'vhigh': 104.40847137227563}),
  2: defaultdict(float,
              {'2': 104.19083659788046,
               '3': 108.17990425127823,
               '4': 105.44920523679052,
               '5more': 109.51433254894404}),
  3: defaultdict(float,
              {'2': 146.94564627393842,
               '4': 142.76098235223327,
               'more': 137.62765000872147}),
  4: defaultdict(float,
              {'big': 143.21818998270382,
               'med': 142.87558501370114,
               'small': 141.2405036384883}),
  5: defaultdict(float,
              {'high': 143.79370975527334,
               'low': 144.26594543988165,

In [52]:

def predict_unsupervised(cf, df, unsupervised_model):
    """
    Predicts the class distribution for a set of
    instances, based on a trained model.
    """
    
    
    
    
    
    
    return


predict_unsupervised(cf, df, um)

,0,1,2,3,4,5,unacc,acc,vgood,good
0,vhigh,vhigh,2,2,small,low,0.013408,0.607227,0.064997,0.314367
1,vhigh,vhigh,2,2,small,med,0.038113,0.127509,0.678461,0.155918
2,vhigh,vhigh,2,2,small,high,0.097038,0.479029,0.082677,0.341256
3,vhigh,vhigh,2,2,med,low,0.266023,0.093445,0.015448,0.625083
4,vhigh,vhigh,2,2,med,med,0.058349,0.344024,0.088702,0.508926
5,vhigh,vhigh,2,2,med,high,0.586590,0.083878,0.047782,0.281750
6,vhigh,vhigh,2,2,big,low,0.072204,0.108967,0.624145,0.194684
7,vhigh,vhigh,2,2,big,med,0.046892,0.219687,0.135917,0.597504
8,vhigh,vhigh,2,2,big,high,0.477617,0.006381,0.049854,0.466148
9,vhigh,vhigh,2,4,small,low,0.005919,0.122032,0.335841,0.536208


In [9]:

def evaluate_unsupervised():
    """
    Evaluates a set of predictions, in an
    unsupervised manner.
    """
    return

Questions (you may respond in a cell or cells below):

1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.
2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.
3. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out (hint: check out numpy.shuffle()) or cross–validation evaluation strategy. How does your estimate of Accuracy change, compared to testing on the training data? Explain why. (The result might surprise you!)
4. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Do you notice any variation in the predictions made by either the supervised or unsupervised NB classifiers? Explain why, or why not.
5. The lecture suggests that deterministically labelling the instances in the initialisation phase of the unsupervised NB classifier “doesn’t work very well”. Confirm this for yourself, and then demonstrate why.
6. Rather than evaluating the unsupervised NB classifier by assigning a class deterministically, instead calculate how far away the probabilistic estimate of the true class is from 1 (where we would be certain of the correct class), and take the average over the instances. Does this performance estimate change, as we alter the number of iterations in the method? Explain why.
7. Explore what causes the unsupervised NB classifier to converge: what proportion of instances change their prediction from the random assignment, to the first iteration? From the first to the second? What is the latest iteration where you observe a prediction change? Make some conjecture(s) as to what is occurring here.

Don't forget that groups of 1 student should respond to question (1), and one other question. Groups of 2 students should respond to question (1), and three other questions. Your responses should be about 100-200 words each.